In [ ]:
from thruster_allocator import thruster_positions, thruster_directions, ThrustAllocator
import matplotlib.pyplot as plt

import numpy as np
%reload_ext autoreload
%autoreload 2

# Thrust Allocator Test

## Thruster Positions and Directions

In [ ]:
%matplotlib qt
ax = plt.figure().add_subplot(projection="3d")
ax.quiver3D(*thruster_positions.T, *thruster_directions.T, length=0.1)
ax.scatter(*(0,0,0))
ax.set_aspect('equal')

## Movement Test

### Thrust Newtons

In [ ]:
%matplotlib qt
thrustAllocator = ThrustAllocator()
outputThrusts = thrustAllocator.getThrusts(
    linear_accelerations = [1.0, 0, 0],
    angular_accelerations = [0, 0, 0]
)

ax = plt.figure().add_subplot(projection="3d")
ax.quiver3D(*thruster_positions.T, *(outputThrusts[:, None] * thruster_directions).T, length=0.02)
ax.scatter(*(0,0,0))
ax.set_aspect('equal')

### Thrust PWMs

In [ ]:
pwms = thrustAllocator.getThrustPWMs(
    linear_accelerations = [0, 0, -3.0],
    angular_accelerations = [0, 0, 0]
)

pwms[np.array([3, 2, 5, 1, 4, 0])]

# Thrust Map

In [ ]:
import pandas as pd
df = pd.read_csv("thrust_map.csv", header=None)
plt.plot(df[1], df[0])
df[0].max()

In [ ]:
thrust_map = pd.read_csv("thrust_map.csv").values
thrust_converted = thrust_map[
    np.searchsorted(thrust_map[:, 1], 172, side="left"), 0
].astype(int)
thrust_converted

# PID Test

In [ ]:
from scipy.integrate import solve_ivp
import random
import matplotlib.pyplot as plt

from PID import PID as PID2

%reload_ext autoreload
%autoreload 2

In [ ]:
time_interval = 0.1

startpoint = 0
setpoint = 5

myPID = PID2(
    Kp=20,
    Ki=0,
    Kd=21,
    sample_time=0.1
    )

def eval_accel(t, u):
    y, v = u
    thrust = myPID.compute(setpoint, y)
    print(thrust)
    accel = constrain(thrust, -4, 4) + random.normalvariate(0, 0.1)
    return [v, accel]

def constrain(val, low, high):
    return min(high, max(low, val))

t_span = [0,16]
sol = solve_ivp(eval_accel, t_span=t_span, y0=[startpoint,0], atol=10, max_step=time_interval)
plt.plot(sol.t, sol.y[0])
plt.axhline(setpoint, color='k')
plt.axhline(setpoint+0.2, color='r', linestyle='--')
plt.axhline(setpoint-0.2, color='r', linestyle='--')